#TODO: Maybe try this - https://youtu.be/0VLAoVGf_74?si=zuJ8AL_wLbsbRdd5
Compare MHA, MQA, MLA (DeepSeek)

## Part 2: Apply TransformerClassifier (Encoder Only) 

In [1]:
from common_utils import *
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from transformer import TransformerClassifierMHA
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score

### Step 1: Prepare Dataset Loader

In [2]:
wassa_dataset = load_dataset("csv", data_files="./dataset/text_emotion.csv")
wassa_dataset = wassa_dataset.rename_column('content', 'text')
wassa_dataset_dict = create_train_validation_test(wassa_dataset['train'])

embedding_matrix = np.load(EMBEDDING_PATH)

with open(WORD2IDX_PATH, "r", encoding="utf-8") as f:
    word2idx = json.load(f)

Train size: 28000
Validation size: 4000
Test size: 8000


In [3]:
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(wassa_dataset_dict['train']['sentiment'])
val_labels = label_encoder.transform(wassa_dataset_dict['validation']['sentiment'])
test_labels = label_encoder.transform(wassa_dataset_dict['test']['sentiment'])
num_classes = len(label_encoder.classes_)
print(f"Number of sentiment classes: {num_classes}")
print(f"Emotion classes: {label_encoder.classes_}")

wassa_labels_dict = {
    'train': train_labels,
    'validation': val_labels,
    'test': test_labels
}

Number of sentiment classes: 13
Emotion classes: ['anger' 'boredom' 'empty' 'enthusiasm' 'fun' 'happiness' 'hate' 'love'
 'neutral' 'relief' 'sadness' 'surprise' 'worry']


In [4]:
wassa_dataloaders_dict = create_dataloaders(wassa_dataset_dict, wassa_labels_dict, word2idx)

Created DataLoaders with 875 training batches, 125 validation batches, and 250 test batches.


### Step 2: Train the TransformerClassifier

In [ ]:
TransformerClassifier_MHA = TransformerClassifierMHA(len(word2idx), 13, 100, 5, 3, 200, 100, 0.1)  
# vocab_size, num_classes, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

TransformerClassifier_MHA.to(device)

embedding_matrix_tensor = torch.FloatTensor(embedding_matrix)
TransformerClassifier_MHA.embedding.weight.data.copy_(embedding_matrix_tensor)
TransformerClassifier_MHA.embedding.weight.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(TransformerClassifier_MHA.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)

Using device: cpu
